In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPool2D, GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras.layers.merge import concatenate
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

In [4]:
def inception_module(layer_in, f1, f2_in, f2_out, f3_in, f3_out, f4_out):
    conv1 = Conv2D(f1, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2_in, (1,1), padding='same', activation='relu')(layer_in)
    conv3 = Conv2D(f2_out, (3,3), padding='same', activation='relu')(conv3)
    conv5 = Conv2D(f3_in, (1,1), padding='same', activation='relu')(layer_in)
    conv5 = Conv2D(f3_out, (5,5), padding='same', activation='relu')(conv5)
    pool = MaxPooling2D((3,3), strides=(1,1), padding='same')(layer_in)
    pool = Conv2D(f4_out, (1,1), padding='same', activation='relu')(pool)
    layer_out = concatenate([conv1, conv3, conv5, pool], axis=-1)
    return layer_out
 
# define model input
visible = Input(shape=(150, 150, 3))
# add inception block 1
layer = inception_module(visible, 64, 96, 128, 16, 32, 32)
# add inception block 1
layer = inception_module(layer, 128, 128, 192, 32, 96, 64)

for fc in [1024, 512, 256]:
    layer = Dense(fc, activation='relu')(layer)
    layer = Dropout(0.5)(layer)
predictions = Dense(5, activation='softmax')(layer)

In [15]:
model = Model(inputs=visible, outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
df = pd.read_csv('train.csv').drop(columns=['Id'])
X = np.array(df.iloc[:, 1:])
y = to_categorical(np.array(df.iloc[:, 0]))

# Convert the training and test images into 3 channels
X = np.dstack([X] * 3)
# Reshape images as per the tensor format required by tensorflow
X = X.reshape(-1, 28, 28, 3)
X = np.asarray([img_to_array(array_to_img(im, scale=False).resize((150,150))) for im in X])
X = X.astype('float32')/255

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5)

(6000, 64, 64, 3)

In [ ]:
model.fit(X_train, y_train, epochs = 25, batch_size = 128, verbose = 1, validation_data=(X_val, y_val))

Train on 48000 samples, validate on 6000 samples
Epoch 1/25
48000/48000 [==============================] - 46s 952us/sample - loss: 0.6341 - accuracy: 0.7706 - val_loss: 49.8181 - val_accuracy: 0.6758
Epoch 2/25
48000/48000 [==============================] - 34s 718us/sample - loss: 0.4306 - accuracy: 0.8296 - val_loss: 0.3576 - val_accuracy: 0.8590
Epoch 3/25
48000/48000 [==============================] - 35s 723us/sample - loss: 0.3535 - accuracy: 0.8574 - val_loss: 0.5823 - val_accuracy: 0.7727
Epoch 4/25
48000/48000 [==============================] - 35s 726us/sample - loss: 0.3280 - accuracy: 0.8677 - val_loss: 0.3106 - val_accuracy: 0.8778
Epoch 5/25
48000/48000 [==============================] - 35s 727us/sample - loss: 0.2984 - accuracy: 0.8780 - val_loss: 0.3533 - val_accuracy: 0.8610
Epoch 6/25
48000/48000 [==============================] - 35s 723us/sample - loss: 0.2818 - accuracy: 0.8856 - val_loss: 0.4454 - val_accuracy: 0.8302
Epoch 7/25
48000/48000 [====================

In [13]:
preds = model.evaluate(X_test, y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

6000/6000 [==============================] - 4s 625us/sample - loss: 0.3802 - accuracy: 0.8853
Loss = 0.38016908676425615
Test Accuracy = 0.88533336


### References 

This notebook presents the ResNet algorithm due to He et al. (2015). The implementation here also took significant inspiration and follows the structure given in the github repository of Francois Chollet: 

- Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun - [Deep Residual Learning for Image Recognition (2015)](https://arxiv.org/abs/1512.03385)
- Francois Chollet's github repository: https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
